In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from mnist import MNIST

In [2]:
mnist_loader = MNIST("../data/mnist/")
mnist_loader.gz = True

In [23]:
images, labels = mnist_loader.load_training()

In [24]:
timages, tlabels = mnist_loader.load_testing()

In [25]:
images = np.concatenate((np.array(images), np.array(timages)))
labels = np.concatenate((np.array(labels), np.array(tlabels)))

In [26]:
images.shape

(70000, 784)

In [27]:
del timages
del tlabels